In [ ]:
# default_exp models

# Models

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import collections
import dataclasses
import enum
import functools
import itertools
import typing
import warnings

import numpy as np
import scipy.stats
import scipy.optimize

import mezzala.blocks
import mezzala.weights
import mezzala.parameters

In [ ]:
#export


@dataclasses.dataclass(frozen=True)
class ScorelinePrediction:
    home_goals: int
    away_goals: int
    probability: float
        

In [ ]:
#export


class Outcomes(enum.Enum):
    HOME_WIN = 'Home win'
    DRAW = 'Draw'
    AWAY_WIN = 'Away win'
    
    def __repr__(self):
        return f"Outcomes('{self.value}')"


@dataclasses.dataclass(frozen=True)
class OutcomePrediction:
    outcome: Outcomes
    probability: float

In [ ]:
#export


def scoreline_to_outcome(home_goals, away_goals):
    if home_goals > away_goals:
        return Outcomes.HOME_WIN
    if home_goals == away_goals:
        return Outcomes.DRAW
    if home_goals < away_goals:
        return Outcomes.AWAY_WIN
    
    
def scorelines_to_outcomes(scorelines):
    return {
        outcome: OutcomePrediction(
            outcome, 
            sum(s.probability for s in scorelines if scoreline_to_outcome(s.home_goals, s.away_goals) == outcome)
        )
        for outcome in Outcomes
    }

In [ ]:
#export

_DEFAULT_DC_BLOCKS = [
    mezzala.blocks.BaseRate(),
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
]


class DixonColes:
    """
    Dixon-Coles models in Python
    """

    def __init__(self, adapter, blocks=_DEFAULT_DC_BLOCKS, weight=mezzala.weights.UniformWeight(), params=None):
        # NOTE: Should params be stored internally as separate lists of keys and values?
        # Then `params` (the dict) can be a property?
        self.params = params
        self.adapter = adapter
        self.weight = weight
        self._blocks = blocks

    def __repr__(self):
        return f'DixonColes(adapter={repr(self.adapter)}, blocks={repr([b for b in self.blocks])}), weight={repr(self.weight)}'

    @property
    def blocks(self):
        # Make sure blocks are always in the correct order
        return sorted(self._blocks, key=lambda x: -x.PRIORITY)

    def home_goals(self, row):
        """ Returns home goals scored """
        return self.adapter.home_goals(row)

    def away_goals(self, row):
        """ Returns away goals scored """
        return self.adapter.away_goals(row)

    def parse_params(self, data):
        """ Returns a tuple of (parameter_names, [constraints]) """
        base_params = [mezzala.parameters.RHO_KEY]
        block_params = list(itertools.chain(*[b.param_keys(self.adapter, data) for b in self.blocks]))
        return (
            block_params + base_params,
            list(itertools.chain(*[b.constraints(self.adapter, data) for b in self.blocks]))
        )

    def _home_terms(self, row):
        return dict(itertools.chain(*[b.home_terms(self.adapter, row) for b in self.blocks]))

    def _away_terms(self, row):
        return dict(itertools.chain(*[b.away_terms(self.adapter, row) for b in self.blocks]))

    # Core methods

    @staticmethod
    def _assign_params(param_keys, param_values):
        return dict(zip(param_keys, param_values))

    def _create_feature_matrices(self, param_keys, data):
        """ Create X (feature) matrices for home and away poisson rates """
        home_X = np.empty([len(data), len(param_keys)])
        away_X = np.empty([len(data), len(param_keys)])
        for row_i, row in enumerate(data):
            home_rate_terms = self._home_terms(row)
            away_rate_terms = self._away_terms(row)
            for param_i, param_key in enumerate(param_keys):
                home_X[row_i, param_i] = home_rate_terms.get(param_key, 0)
                away_X[row_i, param_i] = away_rate_terms.get(param_key, 0)
        return home_X, away_X

    @staticmethod
    def _tau(home_goals, away_goals, home_rate, away_rate, rho):

        tau = np.ones(len(home_goals))
        tau = np.where((home_goals == 0) & (away_goals == 0), 1 - home_rate*away_rate*rho, tau)
        tau = np.where((home_goals == 0) & (away_goals == 1), 1 + home_rate*rho, tau)
        tau = np.where((home_goals == 1) & (away_goals == 0), 1 + away_rate*rho, tau)
        tau = np.where((home_goals == 1) & (away_goals == 1), 1 - rho, tau)

        return tau

    def _log_like(self, home_goals, away_goals, home_rate, away_rate, rho):
        return (
            scipy.stats.poisson.logpmf(home_goals, home_rate) +
            scipy.stats.poisson.logpmf(away_goals, away_rate) +
            np.log(self._tau(home_goals, away_goals, home_rate, away_rate, rho))
        )

    def objective_fn(self, data, home_goals, away_goals, weights, home_X, away_X, rho_ix, xs):
        rho = xs[rho_ix]

        # Parameters are estimated in log-space, but `scipy.stats.poisson`
        # expects real number inputs, so we have to use `np.exp`
        home_rate = np.exp(np.dot(home_X, xs))
        away_rate = np.exp(np.dot(away_X, xs))

        log_like = self._log_like(home_goals, away_goals, home_rate, away_rate, rho)
        pseudo_log_like = log_like * weights
        return -np.sum(pseudo_log_like)

    def fit(self, data, **kwargs):
        param_keys, constraints = self.parse_params(data)

        init_params = (
            # Attempt to initialise parameters from any already-existing parameters
            # This substantially speeds up fitting during (e.g.) backtesting
            np.asarray([self.params.get(p, 0) for p in param_keys])
            # If the model has no parameters, just initialise with 0s
            if self.params
            else np.zeros(len(param_keys))
        )

        # Precalculate the things we can (for speed)

        # Create X (feature) matrices for home and away poisson rates
        home_X, away_X = self._create_feature_matrices(param_keys, data)

        # Get home goals, away goals, and weights from the data
        home_goals, away_goals = np.empty(len(data)), np.empty(len(data))
        weights = np.empty(len(data))
        for i, row in enumerate(data):
            home_goals[i] = self.home_goals(row)
            away_goals[i] = self.away_goals(row)
            weights[i] = self.weight(row)

        # Get the index of the Rho correlation parameter
        rho_ix = param_keys.index(mezzala.parameters.RHO_KEY)

        # Optimise!
        with warnings.catch_warnings():
            # This is a hack
            # Because we haven't properly constrained `rho`, it's possible for 0 or even negative
            # values of `tau` (and therefore invalid probabilities)
            # Ignoring the warnings has little practical impact, since the model
            # will still find the objective function's minimum point regardless
            warnings.simplefilter('ignore')

            estimate = scipy.optimize.minimize(
                lambda xs: self.objective_fn(data, home_goals, away_goals, weights, home_X, away_X, rho_ix, xs),
                x0=init_params,
                constraints=constraints,
                **kwargs
            )

        # Parse the estimates into parameter map
        self.params = self._assign_params(param_keys, estimate.x)

        return self

    def predict_one(self, row, up_to=26):
        scorelines = list(itertools.product(range(up_to), repeat=2))

        home_goals = np.asarray([h for h, a in scorelines])
        away_goals = np.asarray([a for h, a in scorelines])

        param_keys = self.params.keys()
        param_values = np.asarray([v for v in self.params.values()])

        home_X, away_X = self._create_feature_matrices(param_keys, [row])

        home_rate = np.exp(np.dot(home_X, param_values))
        away_rate = np.exp(np.dot(away_X, param_values))
        rho = self.params[mezzala.parameters.RHO_KEY]

        probs = np.exp(self._log_like(home_goals, away_goals, home_rate, away_rate, rho))

        return [ScorelinePrediction(*vals) for vals in zip(home_goals.tolist(), away_goals.tolist(), probs)]

    def predict(self, data, up_to=26):
        scorelines = [self.predict_one(row, up_to=up_to) for row in data]
        return scorelines

In [ ]:
# Dixon-Robinson data = [(time-to-goal, home-or-away-or-no-goal, weight)]
# Likelihood = 
#   Product of:
#     - Likelihood of obs/censored home goal = lambda^(1 if not censored else 0).exp(-lambda.ttg)
#     - Likelihood of obs/censored away goal = <same>

In [ ]:
import json

with open('../data/premier-league-1516-windows.json', 'r') as f:
    windows = json.load(f)

In [ ]:
class TTGSlice(enum.Enum):
    HOME_GOAL = 'Home goal'
    AWAY_GOAL = 'Away goal'
    NO_GOAL = 'No goal'
    
    def __repr__(self):
        return f"TTGSlice('{self.value}')"
    
for w in windows:
    w['window_type'] = TTGSlice(w['window_type'])

In [ ]:
windows[0:3]

[{'match_id': 81,
  'event_type': 'Period end',
  'start_minute': 0,
  'end_minute': 45,
  'window_type': TTGSlice('No goal'),
  'duration': 45,
  'home_team': 'Manchester United',
  'away_team': 'Tottenham',
  'home_score': 0,
  'away_score': 0},
 {'match_id': 81,
  'event_type': 'Period end',
  'start_minute': 45,
  'end_minute': 90,
  'window_type': TTGSlice('No goal'),
  'duration': 45,
  'home_team': 'Manchester United',
  'away_team': 'Tottenham',
  'home_score': 0,
  'away_score': 0},
 {'match_id': 82,
  'event_type': 'Period end',
  'start_minute': 0,
  'end_minute': 45,
  'window_type': TTGSlice('No goal'),
  'duration': 45,
  'home_team': 'Bournemouth',
  'away_team': 'Aston Villa',
  'home_score': 0,
  'away_score': 0}]

In [ ]:
# Get a likelihood for each window
## Likelihood = survival(t)*hazard(t)
##            = lambda^(1 if not censored else 0).exp(-lambda.ttg)
## survival = lambda (...+eps*t)
## hazard   = exp(-lambda*t) (...+ eps*t)

## NB cum-hazard(t) = -log(survival(t))

## => log-likelihood = log(survival(t)) + log(hazard(t))
##                   = log(hazard(t)) - cum-hazard(t) 

In [ ]:
dr_adapter = mezzala.adapters.LambdaAdapter(
    home_team=lambda x: x['home_team'],
    away_team=lambda x: x['away_team'],
    home_goals=lambda x: 1 if x['window_type'] == TTGSlice('Home goal') else 0,
    away_goals=lambda x: 1 if x['window_type'] == TTGSlice('Away goal') else 0,
    time_to_goal=lambda x: max(x['duration'], 1)
)

In [ ]:
_DEFAULT_DR_BLOCKS = [
    mezzala.blocks.BaseRate(),
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
]


class DixonRobinson:
    """
    Dixon-Robinson models in Python
    """
    
    def __init__(self, adapter, blocks=_DEFAULT_DR_BLOCKS, weight=mezzala.weights.UniformWeight(), params=None):
        self.params = params
        self.adapter = adapter
        self.weight = weight
        self._blocks = blocks

    def __repr__(self):
        return f'DixonRobinson(adapter={repr(self.adapter)}, blocks={repr([b for b in self.blocks])}), weight={repr(self.weight)}'

    @property
    def blocks(self):
        # Make sure blocks are always in the correct order
        return sorted(self._blocks, key=lambda x: -x.PRIORITY)

    def parse_params(self, data):
        """ Returns a tuple of (parameter_names, [constraints]) """
        block_params = list(itertools.chain(*[b.param_keys(self.adapter, data) for b in self.blocks]))
        return (
            block_params,
            list(itertools.chain(*[b.constraints(self.adapter, data) for b in self.blocks]))
        )

    def _home_terms(self, row):
        return dict(itertools.chain(*[b.home_terms(self.adapter, row) for b in self.blocks]))

    def _away_terms(self, row):
        return dict(itertools.chain(*[b.away_terms(self.adapter, row) for b in self.blocks]))

    # Core methods

    @staticmethod
    def _assign_params(param_keys, param_values):
        return dict(zip(param_keys, param_values))

    def _create_feature_matrices(self, param_keys, data):
        """ Create X (feature) matrices for home and away poisson rates """
        home_X = np.empty([len(data), len(param_keys)])
        away_X = np.empty([len(data), len(param_keys)])
        for row_i, row in enumerate(data):
            home_rate_terms = self._home_terms(row)
            away_rate_terms = self._away_terms(row)
            for param_i, param_key in enumerate(param_keys):
                home_X[row_i, param_i] = home_rate_terms.get(param_key, 0)
                away_X[row_i, param_i] = away_rate_terms.get(param_key, 0)
        return home_X, away_X

    def _log_like(self, home_goals, away_goals, time_to_event, home_rate, away_rate):
        return (
            # home goal = 1 if censored by a home goal else 0
            # In other words, likelihood = survival function * hazard function
            # where the hazard function is only included when the censoring event
            # (home or away goal) is observed
            # Hazard function = rate
            # Survival function = exp(-rate.t)
            # => log likelihood = ...
            (home_goals*np.log(home_rate) - home_rate*time_to_event) +
            (away_goals*np.log(away_rate) - away_rate*time_to_event)
        )
 
    def objective_fn(self, data, home_goals, away_goals, time_to_event, weights, home_X, away_X, xs):
        # Parameters are estimated in log-space, but `scipy.stats.poisson`
        # expects real number inputs, so we have to use `np.exp`
        home_rate = np.exp(np.dot(home_X, xs))
        away_rate = np.exp(np.dot(away_X, xs))

        log_like = self._log_like(home_goals, away_goals, time_to_event, home_rate, away_rate)
        pseudo_log_like = log_like * weights
        return -np.sum(pseudo_log_like)

    def fit(self, data, **kwargs):
        param_keys, constraints = self.parse_params(data)

        init_params = (
            # Attempt to initialise parameters from any already-existing parameters
            # This substantially speeds up fitting during (e.g.) backtesting
            np.asarray([self.params.get(p, 0) for p in param_keys])
            # If the model has no parameters, just initialise with 0s
            if self.params
            else np.zeros(len(param_keys))
        )

        # Precalculate the things we can (for speed)

        # Create X (feature) matrices for home and away poisson rates
        home_X, away_X = self._create_feature_matrices(param_keys, data)

        # Get home goals, away goals, and weights from the data
        home_goals, away_goals = np.empty(len(data)), np.empty(len(data))
        time_to_event = np.empty(len(data))
        weights = np.empty(len(data))
        for i, row in enumerate(data):
            home_goals[i] = self.adapter.home_goals(row)
            away_goals[i] = self.adapter.away_goals(row)
            time_to_event[i] = self.adapter.time_to_goal(row)
            weights[i] = self.weight(row)

        # Optimise!
        with warnings.catch_warnings():
            # warnings.simplefilter('ignore')

            estimate = scipy.optimize.minimize(
                lambda xs: self.objective_fn(
                    data, home_goals, away_goals, time_to_event, weights, home_X, away_X, xs
                ),
                x0=init_params,
                constraints=constraints,
                **kwargs
            )

        # Parse the estimates into parameter map
        self.params = self._assign_params(param_keys, estimate.x)

        return self

    def predict_one(self, row, up_to=26):
        pass

    def predict(self, data, up_to=26):
        scorelines = [self.predict_one(row, up_to=up_to) for row in data]
        return scorelines

In [ ]:
dr = DixonRobinson(adapter=dr_adapter)
dr.fit(windows)
dr.params

/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in log
/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in multiply
/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in log
/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in multiply


{OffenceParameterKey(label='Everton'): 0.13152621967973194,
 OffenceParameterKey(label='Southampton'): 0.13651274217331383,
 OffenceParameterKey(label='Bournemouth'): -0.11901011860754888,
 OffenceParameterKey(label='Liverpool'): 0.22822769623024555,
 OffenceParameterKey(label='West Ham'): 0.2612933971372183,
 OffenceParameterKey(label='Aston Villa'): -0.697405704057309,
 OffenceParameterKey(label='Sunderland'): -0.10273318685763637,
 OffenceParameterKey(label='Watford'): -0.28875679696487566,
 OffenceParameterKey(label='Chelsea'): 0.11078735462523269,
 OffenceParameterKey(label='Leicester'): 0.2931171217216904,
 OffenceParameterKey(label='Crystal Palace'): -0.2636043933115319,
 OffenceParameterKey(label='Manchester City'): 0.3563834125305029,
 OffenceParameterKey(label='Manchester United'): -0.08591788949291482,
 OffenceParameterKey(label='Swansea'): -0.18636853710014206,
 OffenceParameterKey(label='Norwich'): -0.24596718201808773,
 OffenceParameterKey(label='West Bromwich Albion'): -

In [ ]:
off_params = {k: v for k, v in dr.params.items() if isinstance(k, mezzala.parameters.OffenceParameterKey)}

for k, v in sorted(off_params.items(), key=lambda x: -x[1]):
    print(f'{k.label}:  {np.exp(v):.2f}')

Manchester City:  1.43
Tottenham:  1.35
Leicester:  1.34
West Ham:  1.30
Liverpool:  1.26
Arsenal:  1.24
Southampton:  1.15
Everton:  1.14
Chelsea:  1.12
Manchester United:  0.92
Sunderland:  0.90
Bournemouth:  0.89
Newcastle United:  0.88
Swansea:  0.83
Stoke:  0.82
Norwich:  0.78
Crystal Palace:  0.77
Watford:  0.75
West Bromwich Albion:  0.65
Aston Villa:  0.50


In [ ]:
def_params = {k: v for k, v in dr.params.items() if isinstance(k, mezzala.parameters.DefenceParameterKey)}

for k, v in sorted(def_params.items(), key=lambda x: x[1]):
    print(f'{k.label}:  {np.exp(v):.2f}')

Tottenham:  0.55
Manchester United:  0.56
Arsenal:  0.56
Leicester:  0.62
Manchester City:  0.69
Southampton:  0.70
West Bromwich Albion:  0.75
Crystal Palace:  0.78
Swansea:  0.79
Watford:  0.80
Liverpool:  0.84
West Ham:  0.84
Chelsea:  0.87
Everton:  0.92
Stoke:  0.92
Sunderland:  1.02
Newcastle United:  1.06
Norwich:  1.08
Bournemouth:  1.12
Aston Villa:  1.18


In [ ]:
# With scorelines
def classify_scorelines(row):
    scoreline = (row['home_score'], row['away_score'])
    if scoreline == (0, 0):
        return ((0, 0), 0.0)
    if scoreline in [(1, 0), (0, 1), (1, 1)]:
        return (scoreline, 1.0)
    
    delta = row['home_score'] - row['away_score']
    if delta == 0:
        return ('Drawing with 2+ goals', 1.0)
    if delta >= 1:
        return ('Home leading with 2+ goals', 1.0)
    if delta <= -1:
        return ('Away leading with 2+ goals', 1.0)


dr_scoreline_blocks = _DEFAULT_DR_BLOCKS + [
    # Scorelines
    mezzala.blocks.HomeBlock(lambda x: (('Home scoreline', classify_scorelines(x)[0]), classify_scorelines(x)[1])),
    mezzala.blocks.AwayBlock(lambda x: (('Away scoreline', classify_scorelines(x)[0]), classify_scorelines(x)[1])),
#     # Time period
#     mezzala.blocks.HomeBlock(lambda x: (('Home time bucket', (x['start_minute'] + x['duration']/2)//10), 1)),
#     mezzala.blocks.AwayBlock(lambda x: (('Away time bucket', (x['start_minute'] + x['duration']/2)//10), 1)),
]

dr = DixonRobinson(blocks=dr_scoreline_blocks, adapter=dr_adapter)
dr.fit(windows)
dr.params

/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in log
/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in multiply
/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in log
/Users/ben/code/mezzala/venv/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in multiply


{OffenceParameterKey(label='Everton'): 0.12370727704213759,
 OffenceParameterKey(label='Southampton'): 0.14610316042275534,
 OffenceParameterKey(label='Bournemouth'): -0.16419519594332033,
 OffenceParameterKey(label='Liverpool'): 0.22772474956996924,
 OffenceParameterKey(label='West Ham'): 0.2282510422646767,
 OffenceParameterKey(label='Aston Villa'): -0.7154972004201889,
 OffenceParameterKey(label='Sunderland'): -0.12506572570642996,
 OffenceParameterKey(label='Watford'): -0.3046214786784996,
 OffenceParameterKey(label='Chelsea'): 0.11222758499431107,
 OffenceParameterKey(label='Leicester'): 0.3142616341818241,
 OffenceParameterKey(label='Crystal Palace'): -0.21937457420225187,
 OffenceParameterKey(label='Manchester City'): 0.350938231759438,
 OffenceParameterKey(label='Manchester United'): -0.05986254465339363,
 OffenceParameterKey(label='Swansea'): -0.19725944511589363,
 OffenceParameterKey(label='Norwich'): -0.2557741306335544,
 OffenceParameterKey(label='West Bromwich Albion'): -0

In [ ]:
scoreline_params = {k: v for k, v in dr.params.items() if isinstance(k, tuple) and ('scoreline' in k[0])}

for k, v in sorted(scoreline_params.items(), key=lambda x: x[1]):
    print(f'{k}:  {np.exp(v):.2f}')

('Away scoreline', (0, 0)):  1.00
('Home scoreline', (0, 0)):  1.00
('Home scoreline', (1, 1)):  1.04
('Home scoreline', 'Home leading with 2+ goals'):  1.14
('Away scoreline', 'Away leading with 2+ goals'):  1.17
('Away scoreline', (1, 0)):  1.23
('Home scoreline', (1, 0)):  1.23
('Away scoreline', (1, 1)):  1.27
('Away scoreline', (0, 1)):  1.30
('Home scoreline', (0, 1)):  1.36
('Away scoreline', 'Drawing with 2+ goals'):  1.47
('Home scoreline', 'Away leading with 2+ goals'):  1.49
('Away scoreline', 'Home leading with 2+ goals'):  1.58
('Home scoreline', 'Drawing with 2+ goals'):  1.70


In [ ]:
off_params = {k: v for k, v in dr.params.items() if isinstance(k, mezzala.parameters.OffenceParameterKey)}

for k, v in sorted(off_params.items(), key=lambda x: -x[1]):
    print(f'{k.label}:  {np.exp(v):.2f}')

Manchester City:  1.42
Tottenham:  1.40
Leicester:  1.37
West Ham:  1.26
Liverpool:  1.26
Arsenal:  1.25
Southampton:  1.16
Everton:  1.13
Chelsea:  1.12
Manchester United:  0.94
Sunderland:  0.88
Newcastle United:  0.88
Bournemouth:  0.85
Swansea:  0.82
Stoke:  0.81
Crystal Palace:  0.80
Norwich:  0.77
Watford:  0.74
West Bromwich Albion:  0.65
Aston Villa:  0.49
